# WikipediaRetriever

>[Wikipedia](https://wikipedia.org/) 是一个多语言的免费在线百科全书，由一群称为维基百科人的志愿者通过开放协作编写和维护，并使用一个名为 MediaWiki 的维基编辑系统。`Wikipedia` 是历史上规模最大、阅读最广泛的参考工具书。

本笔记本展示了如何从 `wikipedia.org` 中检索维基页面，并将其转换为下游使用的 [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 格式。

### 集成详情

import {ItemTable} from "@theme/FeatureTables";

<ItemTable category="external_retrievers" item="WikipediaRetriever" />

## 设置
要启用对单个工具的自动化跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此集成位于 `langchain-community` 包中。我们还需要安装 `wikipedia` Python 包本身。

In [ ]:
%pip install -qU langchain_community wikipedia

## 实例化

现在我们可以实例化我们的检索器：

`WikipediaRetriever` 参数包括：
- 可选 `lang`：默认值为 "en"。用它来搜索维基百科的特定语言部分
- 可选 `load_max_docs`：默认值为 100。用它来限制下载文档的数量。下载全部 100 篇文档需要时间，所以在实验中请使用较小的数字。目前有一个硬性限制为 300。
- 可选 `load_all_available_meta`：默认值为 False。默认情况下只下载最重要的字段：`Published`（文档发布/最后更新日期）、`title`、`Summary`。如果设置为 True，则也会下载其他字段。

`get_relevant_documents()` 有一个参数 `query`：用于在维基百科中查找文档的自由文本

In [1]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

## 用法

In [2]:
docs = retriever.invoke("TOKYO GHOUL")

In [3]:
print(docs[0].page_content[:400])

Tokyo Ghoul (Japanese: 東京喰種（トーキョーグール）, Hepburn: Tōkyō Gūru) is a Japanese dark fantasy manga series written and illustrated by Sui Ishida. It was serialized in Shueisha's seinen manga magazine Weekly Young Jump from September 2011 to September 2014, with its chapters collected in 14 tankōbon volumes. The story is set in an alternate version of Tokyo where humans coexist with ghouls, beings who loo


## 在链中使用
与其他检索器一样，`WikipediaRetriever` 可以通过 [链](/docs/how_to/sequence/) 整合到 LLM 应用中。

我们需要一个 LLM 或聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based only on the context provided.
    Context: {context}
    Question: {question}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
chain.invoke(
    "Who is the main character in `Tokyo Ghoul` and does he transform into a ghoul?"
)

'The main character in Tokyo Ghoul is Ken Kaneki, who transforms into a ghoul after receiving an organ transplant from a ghoul named Rize.'

## API 参考

有关所有 `WikipediaRetriever` 功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.wikipedia.WikipediaRetriever.html#langchain-community-retrievers-wikipedia-wikipediaretriever)。